# Optimización y Procesos Estocásticos

## Análisis de Texto

El propósito de esta sección es analizar una colección de documentos de textos y clasificarlos de acuerdo a un conjunto establecido de categorias. 

En el siguiente ejemplo se utiliza el conjunto 20 Newsgroups para entrenar un clasificador.  

## Ejemplo

### Cargar el dataset 20 newsgroups

En conjunto de datos está dividido en 20 categorias. Para facilitar la ejecución, solo se cargan las siguientes 4 categorias:

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

Ahora se cargan las categorias seleccionadas a partir del conjunto de datos de entrenamiento `train`:

In [ ]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

es posible imprimir el nombre de las categorias mediante el atributo `target_names`:

In [ ]:
twenty_train.target_names

los datos están almacenados en el atributo `data`

In [ ]:
len(twenty_train.data)

cada dato está almacenado en un archivo:

In [ ]:
len(twenty_train.filenames)

es posible imprimir las 3 primeras líneas del primer registro:

In [ ]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

y el nombre de la categoría a la que pertenece:

In [ ]:
print(twenty_train.target_names[twenty_train.target[0]])

los algoritmos de aprendizaje supervisado requieren una etiqueta de categoría para cada documento en el conjunto de entrenamiento. 

Para mayor eficiencia (velocidad de acceso y almacenamiento), las categorias se guardan como un arreglo de índices que corresponden con el nombre de la categoría en la lista `target_names`:

In [ ]:
twenty_train.target[:10]

además, es posible obtener el nombre de las categorias por medio de:

In [ ]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

### Extraer características de los archivos de texto

Para aplicar aprendizaje de máquina se debe convertir el contenido de texto en vectores de características numéricas.

#### Bolsa de palabras (Bag of words)

La bolsa de palabras es una forma intuitiva de representar vectores de características. Para constrir una bolsa de palabras, se debe:

1. Asignar un entero a cada palabra presente en el documento del conjunto de entrenamiento (Ej: crear diccionario)

1. Para cada documento `#i`, contar el número de ocurrencias de cada palabra `w` y almacenarla en `X[i, j]` como el valor de la característica `#j`, donde `j` es el índice de la palabra `w` en el diccionario.

#### Tokenizar texto

Procesar texto, tokenizar y filtrar palabras son funciones incluidas  en el módulo que permite construir el diccionario de características.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

`CountVetorizer()` soporta el conteo de N-gramas de palabras o caracteres consecutivos. Una vez ajustado, el vectorizador construye un diccionario de índices de características.

In [ ]:
count_vect.vocabulary_.get(u'algorithm')

el valor del índice de una palabra en el vocabulario está relacionado con su frecuencia en el corpus de entrenamiento.

#### Ocurrencias y frecuencias

El conteo de ocurrencias es un buen inicio pero presenta un problema: los documentos grandes tienen promedios mayores de conteos en relación con documentos más cortos; incluso así traten sobre el mismo tema.

Para evitar este problema se divide el conteo ocurrencias entre el número total de palabras. Esta característica es llamada *Frecuencia de Término* `tr`:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

X_train_tf = tf_transformer.transform(X_train_counts)

X_train_tf.shape

Otro refinamiento de `tr` consite en reducir la escala de los pesos de las palabras presentes en varios documentos y son por lo tanto menos informativas que equellas que ocurren una porción pequeña del corpus.

La reducción de escala es llamada *Frecuencia de Término – Frecuencia Inversa de Documento* `tr-idf`:

In [ ]:
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_train_tfidf.shape

### Entrenar un clasificador

A partir de las características, es posible entrenar un clasificador para predecir la categoría de un post. Para iniciar se utilizará el clasificador *Bayesiano ingenuo*:

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

Para predecir la salida de un documento nuevo, se debe extraer el vector de características usando la misma secuencia de comandos: 

In [ ]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']

In [ ]:
X_new_counts = count_vect.transform(docs_new)
print (X_new_counts)

In [ ]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
print (X_new_tfidf)

In [ ]:
predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

## Referencias

* http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html